# 🔐 Authenticate to Lakebase Instance with Azure EntraID Service Principal

## 🧱 Step 1: Register an App in Entra ID
- Go to **Azure Portal** → **Microsoft Entra ID** (formerly Azure Active Directory).
- In the left menu, select **App registrations → + New registration**.
- Give it a name (e.g., `databricks-spn`).
- Choose **Accounts in this organizational directory only (default)**.
- Click **Register**.
- Once registered, note down:
  - **Application (client) ID** → your `azure_client_id`
  - **Directory (tenant) ID** → your `azure_tenant_id`

---

## 🔑 Step 2: Create a Client Secret
- In the left sidebar, go to **Certificates & secrets → Client secrets**.
- Click **+ New client secret**.
- Enter a description (e.g., *for Databricks SDK*).
- Choose an expiration period (1 year, 2 years, or custom).
- Click **Add**.
- Copy the **Value** immediately — this is your `azure_client_secret`.
  - ⚠️ You won’t be able to view it again after leaving the page.

---

## 🧰 Step 3: Assign Permissions to Databricks Workspace
You must authorize the Azure service principal to access the Databricks workspace.

- In **Databricks Workspace**, go to:
  - **Workspace settings → Identity and access → Service principals**
- Click **Add Service Principal → Add New**.
- Select **Microsoft Entra ID managed**  
  *(Your service principal will link with an existing Microsoft Entra ID app and be managed externally.)*
- Enter:
  - **Microsoft Entra Application ID**
  - **Service Principal Name**
- Click **Create service principal**.
- In the service principal configuration page:
  - Enable **Databricks SQL access**.
  - Enable **Workspace access** entitlements.

---

## 🧩 Step 4: Assign Permissions to Lakebase Instance
- Go to your **Lakebase Database instance** page.
- Navigate to **Permissions**.
- Click **Add PostgreSQL role**.
- Assign the `databricks_superuser` role to the Azure service principal.


In [0]:
%pip install --upgrade databricks-sdk
%pip install sqlalchemy==1.4 psycopg[binary]
%restart_python

In [0]:
from databricks.sdk import WorkspaceClient

# Azure AD Service Principal authentication
w = WorkspaceClient(
    host="https://<databricks-instance-url>",
    auth_type="azure-client-secret",
    azure_client_id="<azure-client-id>",
    azure_client_secret="<azure-client-secret>",
    azure_tenant_id="<azure-tenant-id>"
)

# Verify connection
me = w.current_user.me()
print(f"Authenticated as: {me.user_name}")


In [0]:
import uuid
instance_name = "<postgres-instance-name>"
instance = w.database.get_database_instance(name=instance_name)
cred = w.database.generate_database_credential(request_id=str(uuid.uuid4()), instance_names=[instance_name])
print(instance)

In [0]:
from sqlalchemy import create_engine, text

from databricks.sdk import WorkspaceClient
import uuid

user = w.current_user.me().user_name
host = instance.read_write_dns
port = 5432
database = "databricks_postgres"
password = cred.token

connection_pool = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}?sslmode=require")

with connection_pool.connect() as conn:
    result = conn.execute(text("SELECT version()"))
    for row in result:
        print(f"Connected to PostgreSQL database. Version: {row}")